## Handle Imports

In [186]:
import numpy as np
import pandas as pd
import csv
import statistics as stats

# Upload methods

### Load Countries

In [187]:
def load_countries() -> dict:
    countries = {}

    with open("countries.csv") as file:
        reader = csv.DictReader(file)
        for row in reader:
            key = row["Country"]
            countries[key] = {name: int(value) for name, value in row.items() if name != "Country"}

    return countries

### Load Resources

In [188]:
def load_resources() -> dict:
    resources = {}

    with open("resources.csv") as file:
        reader = csv.DictReader(file)
        for row in reader:
            key = row["Resource"]
            resources[key] = {name: float(value) if name == "Weight" else value for name, value in row.items() if name != "Resource"}

    return resources

## Upload Countries

In [189]:
countryList = load_countries()

In [190]:
resourceList = load_resources()

In [191]:
print(countryList)

{'Atlantis': {'R1': 20, 'R2': 12, 'R3': 10, 'R4': 20, 'R5': 10, 'R6': 5, 'R7': 9, 'R8': 40, 'R9': 1, 'R10': 24}, 'Dinotopoia': {'R1': 15, 'R2': 0, 'R3': 2, 'R4': 60, 'R5': 0, 'R6': 0, 'R7': 10, 'R8': 4, 'R9': 2, 'R10': 56}, 'Erewhon': {'R1': 21, 'R2': 2, 'R3': 3, 'R4': 40, 'R5': 0, 'R6': 30, 'R7': 26, 'R8': 2, 'R9': 0, 'R10': 43}, "King's Landing": {'R1': 80, 'R2': 3, 'R3': 4, 'R4': 30, 'R5': 0, 'R6': 15, 'R7': 35, 'R8': 3, 'R9': 0, 'R10': 23}, 'The Vale': {'R1': 43, 'R2': 1, 'R3': 5, 'R4': 10, 'R5': 0, 'R6': 12, 'R7': 30, 'R8': 4, 'R9': 0, 'R10': 10}}


In [192]:
print(resourceList)

{'R1': {'Weight': 0.0, 'Notes': 'analog to population'}, 'R2': {'Weight': 0.0, 'Notes': 'analog to metalic elements'}, 'R3': {'Weight': 0.0, 'Notes': 'analog to timber'}, 'R4': {'Weight': 0.2, 'Notes': 'analog to metallic alloys'}, 'R5': {'Weight': 0.5, 'Notes': 'analog to electronics'}, 'R6': {'Weight': 0.8, 'Notes': 'analog to housing'}, 'R7': {'Weight': -0.5, 'Notes': 'waste'}, 'R8': {'Weight': -0.8, 'Notes': 'waste'}, 'R9': {'Weight': -0.4, 'Notes': 'waste'}, 'R10': {'Weight': -0.2, 'Notes': 'waste'}}


### Set to Pandas Dataframe

In [193]:
countrydf = pd.DataFrame(countryList).transpose()

In [194]:
resourcedf = pd.DataFrame(resourceList).transpose()

In [195]:
countrydf

,R1,R2,R3,R4,R5,R6,R7,R8,R9,R10
Atlantis,20,12,10,20,10,5,9,40,1,24
Dinotopoia,15,0,2,60,0,0,10,4,2,56
Erewhon,21,2,3,40,0,30,26,2,0,43
King's Landing,80,3,4,30,0,15,35,3,0,23
The Vale,43,1,5,10,0,12,30,4,0,10


In [196]:
resourcedf

,Weight,Notes
R1,0.0,analog to population
R2,0.0,analog to metalic elements
R3,0.0,analog to timber
R4,0.2,analog to metallic alloys
R5,0.5,analog to electronics
R6,0.8,analog to housing
R7,-0.5,waste
R8,-0.8,waste
R9,-0.4,waste
R10,-0.2,waste


# State Quality Evaluation

### Check Country List

In [197]:
for key in countryList:
    print(key, '->', countryList[key])

Atlantis -> {'R1': 20, 'R2': 12, 'R3': 10, 'R4': 20, 'R5': 10, 'R6': 5, 'R7': 9, 'R8': 40, 'R9': 1, 'R10': 24}
Dinotopoia -> {'R1': 15, 'R2': 0, 'R3': 2, 'R4': 60, 'R5': 0, 'R6': 0, 'R7': 10, 'R8': 4, 'R9': 2, 'R10': 56}
Erewhon -> {'R1': 21, 'R2': 2, 'R3': 3, 'R4': 40, 'R5': 0, 'R6': 30, 'R7': 26, 'R8': 2, 'R9': 0, 'R10': 43}
King's Landing -> {'R1': 80, 'R2': 3, 'R3': 4, 'R4': 30, 'R5': 0, 'R6': 15, 'R7': 35, 'R8': 3, 'R9': 0, 'R10': 23}
The Vale -> {'R1': 43, 'R2': 1, 'R3': 5, 'R4': 10, 'R5': 0, 'R6': 12, 'R7': 30, 'R8': 4, 'R9': 0, 'R10': 10}


### Set Maslow Constants

In [208]:
#different resource levels
levelOneResources = {'R1': 10, 'R3': 20}
levelTwoResources = {'R2': 20, 'R4': 5} 
levelThreeResources = {'R5': 20} 
levelFourResources = {'R6': 35}
levelFiveResources = {'R7': 35, 'R8': 20, 'R9': 10, 'R10': 25}

#list of levels
levList = [levelOneResources, levelTwoResources, levelThreeResources, levelFourResources, levelFiveResources]

In [209]:
print(levList)

[{'R1': 10, 'R3': 20}, {'R2': 20, 'R4': 5}, {'R5': 20}, {'R6': 35}, {'R7': 35, 'R8': 20, 'R9': 10, 'R10': 25}]


### Level Function

In [217]:
#pandas version
def leveldf(df, country, level, resources):
    
    levelSat = True
    mult = []
    average = 0

    for key, value in levList[level-1].items():
        countryVal = df.loc[country, key]
        mult.append(countryVal/value)
        if countryVal < value:
            levelSat = False
    
    average = stats.mean(mult)
    
    if not levelSat:
        average = average/4
        
    return average

In [218]:
num = leveldf(countrydf, 'Atlantis', 1, levList)
num

0.3125

In [219]:
#dictionary version (doesn't work)
def level(country, level):

    levelSat = True
    mult = []
    average = 0
    
    #go through each resource of the country
    for key, value in country:
        if key in level.keys():
            mult.append(value/level.keys()[0])
            if value < level.keys()[0]:
                levelSat = False
    
    average = mult.sum()/mult.len()
    
    if not levelSat:
        average = average/4
        
    return average

## Maslow Function

In [222]:
def maslow(df, country, level, resources):
    
    maslowList = []
    
    for num in range(1, level+1):
        levValue = leveldf(countrydf, country, num, levList)
        maslowList.append(levValue)
            
    return maslowList

In [223]:
maslowL = maslow(countrydf, 'The Vale', 5, levList)

In [224]:
print(maslowL)

[0.56875, 0.25625, 0.0, 0.08571428571428572, 0.09107142857142857]


In [225]:
def maslowHeuristicVal(lst):
    return stats.mean(maslowL)

In [226]:
masVal = maslowHeuristicVal(maslowL)

In [227]:
print(masVal)

0.20035714285714284
